W zadaniu 1 regresja służyła de facto do przeprowadzenia klasyfikacji. Modele regresyjne mają też jednak inne zastosowanie - można przy ich użyciu przewidywać wartości ciągłe. W tym celu w wektorze y nie zawieramy klas, tylko wartości jakiegoś parametru lub wielkości fizycznej, które mogą zmieniać się w sposób ciągły i uzależnione są od wartości zmiennych niezależnych (opisujących obiekt). Nauczony model ma za zadanie przewidzieć, jaka wartość pojawi się w zmiennej y, jeżeli na wejściu dostanie ciąg opisujących obiekt cech.

Przykładowy model regresyjny posłuży nam do wyznaczania, jaka jest częstotliwość tonu podstawowego (F0) głosu w nagraniach samogłosek o przedłużonej fonacji emitowanyc na różnej wysokości.

W wektorze X znajdują się cechy wyekstrahowane przy użyciu biblioteki opensmile, w wektorze y - wartości F0 w danym nagraniu.

In [111]:
import numpy as np
from sklearn.linear_model import BayesianRidge
from sklearn.metrics import mean_absolute_error, make_scorer, mean_squared_error, accuracy_score, confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, f_regression
import matplotlib.pyplot as plt
import optuna
from sklearn.model_selection import cross_validate, KFold
import pickle
from sklearn.pipeline import make_pipeline

In [112]:
#wczytywanie danych
X = np.load('opensmile_feats.npy') #macierz cech - jednej wiersz = jeden obiekt
y = np.load('fundamental_frequencies.npy') #labele
print(y.shape)

(909,)


In [113]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42) #podział na zbiory

In [114]:
scaler = StandardScaler().fit(X_train) #standaryzacja
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

Ponieważ liczba cech jest duża, regresor albo trenowałby się długo, albo w ogóle nie byłby w stanie poprawnie nauczyć się określać F0. By rozwiązać ten problem, zaczniemy od redukcji wymiarowości - przeprowadź redukcję funkcją SelectKBest.

Liczba cech, którą należy uzyskać po redukcji jest już określona i została dobrana losowo - jej optymalizacja będzie na końcu zadania.

In [115]:
liczba_cech = 100
selecter = SelectKBest(score_func=f_regression, k=liczba_cech)
X_train_dim_reduced = selecter.fit_transform(X_train_scaled, y_train)
X_test_dim_reduced = selecter.fit_transform(X_test_scaled, y_test)


c:\Users\jakub\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:289: RuntimeWarning: invalid value encountered in true_divide
  correlation_coefficient /= X_norms
c:\Users\jakub\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:289: RuntimeWarning: invalid value encountered in true_divide
  correlation_coefficient /= X_norms


Po redukcji wymiarowości możemy przystąpić do uczenia modelu i predykcji. W wyniku predykcji powinniśmy dostać wartości F0, które wg modelu odpowiadają analizowanym nagraniom.

Użyjemy algorytmu BayesianRidge, który jest dość podobny do klasycznej regresji liniowej. Różnica jest taka, że w regresji liniowej współczynniki przy kolejnych składowych (odpowiadających zmiennym zależnym) wyrażone są konkretną wartością liczbową, natomiast w regresji bayesowskiej estymuje się rozkłady prawdopodobieństwa tych współczynników. Ponadto, ponieważ jest to regresja typu ridge, to wykorzystuje normę L2 podczas regularyzacji.

In [116]:
#X_train_dim_reduced, X_test_dim_reduced - macierze cech po redukcji wymiarowości
ridge_reg = BayesianRidge()
ridge_reg.fit(X_train_dim_reduced, y_train)
preds_test = ridge_reg.predict(X_test_dim_reduced)

Średni błąd bezwględny (MAE) wylicza się jako średnią wartość |y_test - preds_test|. Im mniejsza wartość MAE, tym lepsze predykcje.

Uzyskana wartość MAE może być interpretowana jako średnia liczba Hz, o jaką pomylił się regresor przewidując F0.

In [117]:
print(mean_absolute_error(y_test, preds_test))

85.47045735182812


Model można zoptymalizować, żeby uzyskać lepsze wyniki. Należy zmienić metrykę, która będzie wykorzystywana do oceny modelu podczas optymalizacji, czyli scoring. Ponadto trzeba zmienić direction='maximize' na direction='minimize', ponieważ tym razem zależy nam na jak najmniejszej wartości metryki, a nie jak największej. Powinniśmy również zmienić StratifiedKFold na KFold, ponieważ stratyfikacja ma sens tylko w zadaniu klasyfikacyjnym.

Pozostały kod powinien wyglądać analogicznie jak w poprzednich zadaniach.

In [118]:
scoring = {'mae': make_scorer(mean_absolute_error)}

In [119]:
def objective(trial, model, space, X, y):
    model_space = get_space(trial)

    mdl = model(**model_space)
    scores = cross_validate(mdl, X, y, scoring=scoring, cv=KFold(n_splits=5), return_train_score=True)

    return np.mean(scores['test_mae'])

Hiperparametry, które należy zoptymalizować i powinny znaleźć się w funkcji get_space to:
- alpha_1
- alpha_2
- lambda_1
- lambda_2

Wszystkie powinny przyjmować wartości ciągłe z przedziału [0,1].

In [120]:
model = BayesianRidge
def get_space(trial):
    space =  {"alpha_1": trial.suggest_float('alpha_1', 0.0000001, 0.0001),
            "alpha_2": trial.suggest_float('alpha_2', 0.0000001, 0.0001),
            "lambda_1": trial.suggest_float('lambda_1', 0.0000001, 0.0001),
            "lambda_2": trial.suggest_float('lambda_2', 0.0000001, 0.0001)}
    return space
trials = 30

Na początku najlepiej pracować na małej liczbie triali - jak kod już zadziała, trzeba będzie ją zwiększyć. 5 prób to za mało, żeby uzyskać dobre wyniki przy optymalizacji kilku hiperparametrów.

In [121]:
study = optuna.create_study(direction='minimize')
study.optimize(lambda x: objective(x, model, get_space, X_train_dim_reduced, y_train), n_trials=trials)

[I 2022-12-19 22:26:52,968] A new study created in memory with name: no-name-2a747b57-86a0-4381-8180-0e9d85b7a09a
[I 2022-12-19 22:26:53,162] Trial 0 finished with value: 35.55002750745494 and parameters: {'alpha_1': 9.563923945476126e-05, 'alpha_2': 3.415930780980981e-05, 'lambda_1': 8.709620120553179e-05, 'lambda_2': 2.6166461454453323e-06}. Best is trial 0 with value: 35.55002750745494.
[I 2022-12-19 22:26:53,369] Trial 1 finished with value: 35.55002765914025 and parameters: {'alpha_1': 7.832963917903754e-05, 'alpha_2': 6.0830328706319114e-05, 'lambda_1': 5.280873021634457e-05, 'lambda_2': 4.3318833017603926e-05}. Best is trial 0 with value: 35.55002750745494.
[I 2022-12-19 22:26:53,581] Trial 2 finished with value: 35.55002770128171 and parameters: {'alpha_1': 1.0402188845213296e-05, 'alpha_2': 1.2129754924236187e-05, 'lambda_1': 3.711885190188406e-05, 'lambda_2': 7.735334130297255e-05}. Best is trial 0 with value: 35.55002750745494.
[I 2022-12-19 22:26:53,825] Trial 3 finished wi

Powtórz optymalizację zmieniając metrykę z MAE na błąd średniokwadratowy MSE (mean_squared_error). Czy wyniki różnią się? Jak myślisz, różniłyby się, gdyby liczba triali była większa, czy powinny zbiegać do tej samej wartości?

In [122]:
print('params: ', study.best_params)

ridge_reg = model(**study.best_params)
ridge_reg.fit(X_train_dim_reduced, y_train)
preds = ridge_reg.predict(X_test_dim_reduced)
pickle.dump(ridge_reg, open('ridge_F0_model_mse', 'wb+'))
print('test MAE: ', mean_absolute_error(y_test, preds))
print('test MSE: ', mean_squared_error(y_test, preds))

params:  {'alpha_1': 2.764471367000136e-06, 'alpha_2': 9.802976037105732e-05, 'lambda_1': 9.998279160117151e-05, 'lambda_2': 9.165240757965775e-05}
test MAE:  85.47032063522393
test MSE:  10036.970005818124


Na razie liczba cech wybranych na etapie redukcji wymiarowości wynosi 100 i została wybrana losowo. Spróbuj przeprowadzić optymalizację modelu w taki sposób, by liczba cech też była optymalizowana.

Podpowiedź: jeżeli dodasz dodatkowy parametr "k" do get_space, to trzeba go będzie usunąć przed utworzeniem modelu w funkcji objective (del model_space["k"]) oraz utworzyć w tej funkcji pipeline, który będzie się składał z selektora SelectKBest o zoptymalizowanej liczbie k oraz z modelu.

In [123]:
def objective(trial, model, space, X, y):

    model_space = get_space(trial)
    k = model_space['k']
    del model_space['k']
    mdl = make_pipeline(SelectKBest(score_func=f_regression,k=k) , model(**model_space))
    scores = cross_validate(mdl, X, y, scoring=scoring, cv=KFold(n_splits=5), return_train_score=True)

    return np.mean(scores['test_mae'])

model = BayesianRidge
def get_space(trial):
    space =  {"alpha_1": trial.suggest_float('alpha_1', 0.0000001, 0.0001),
            "alpha_2": trial.suggest_float('alpha_2', 0.0000001, 0.0001),
            "lambda_1": trial.suggest_float('lambda_1', 0.0000001, 0.0001),
            "lambda_2": trial.suggest_float('lambda_2', 0.0000001, 0.0001),
            "k": trial.suggest_int('k', 10, 1000)}
    return space
trials = 30

In [124]:
study = optuna.create_study(direction='minimize')
study.optimize(lambda x: objective(x, model, get_space, X_train_scaled, y_train), n_trials=trials)

[I 2022-12-19 22:27:01,157] A new study created in memory with name: no-name-87307fc1-9607-4b66-991a-3dbe5658672e
c:\Users\jakub\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:289: RuntimeWarning: invalid value encountered in true_divide
  correlation_coefficient /= X_norms
c:\Users\jakub\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:289: RuntimeWarning: invalid value encountered in true_divide
  correlation_coefficient /= X_norms
c:\Users\jakub\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:289: RuntimeWarning: invalid value encountered in true_divide
  correlation_coefficient /= X_norms
c:\Users\jakub\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:289: RuntimeWarning: invalid value encountered in true_divide
  correlation_coefficient /= X_

In [125]:

k = study.best_params['k']
params = study.best_params
del params['k']
print('params: ', params)
ridge_reg = make_pipeline(SelectKBest(score_func=f_regression, k=k) , model(**params))
ridge_reg.fit(X_train_scaled, y_train)
preds = ridge_reg.predict(X_test_scaled)
pickle.dump(ridge_reg, open('ridge_F0_model_mse', 'wb+'))
print('test MAE: ', mean_absolute_error(y_test, preds))
print('test MSE: ', mean_squared_error(y_test, preds))

c:\Users\jakub\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:289: RuntimeWarning: invalid value encountered in true_divide
  correlation_coefficient /= X_norms


params:  {'alpha_1': 5.0202817745971345e-05, 'alpha_2': 5.2417119821286434e-05, 'lambda_1': 2.63103103394604e-05, 'lambda_2': 8.11466238687345e-05}
test MAE:  24.24503464225252
test MSE:  1070.2719231778497
